In [1]:
import numpy as np
import pandas as pd

In [2]:
shelter = pd.read_csv("shelterinplace.csv") # scraped via "webscraping.ipynb" notebook, which pulls from NYTimes
data_counties = pd.read_csv("us-counties.csv") # pulled from NYTimes github page, https://github.com/nytimes/covid-19-data
city_county = pd.read_csv("city_county.csv") # Manually created to handle city/county discrepancies between two files

In [3]:
# Convert dates to datetime for comparison
shelter['date_initiated'] = pd.to_datetime(shelter['date'])
data_counties['date'] = pd.to_datetime(data_counties['date'])
shelter.drop('date',axis=1, inplace=True)

# Replace cities with counties
df_join = shelter.merge(city_county, how='left', left_on=['state','area'], right_on=['state','area'])
shelter.loc[df_join['county'].notnull(),'area'] = df_join.loc[df_join['county'].notnull(),'county']

# Remove the " County" suffix from area
shelter['area'] = shelter['area'].str.replace(" County","")

In [8]:
# Join county-level data
county_level = shelter.query('area !="ALL"')

# Merge to county data set
final = data_counties.merge(county_level.drop(['population'],axis=1), how="left", left_on=["county","state"],right_on=["area","state"])

In [9]:
# Join state-level data
state_level = shelter.query('area == "ALL"')

state_subset = final.merge(state_level.drop(['area','population'],axis=1), how="left", left_on="state",right_on="state")

final.loc[final["type"].isnull(),"type"] = state_subset.loc[state_subset['area'].isnull(),'type_y']
final.loc[final["date_initiated"].isnull(),"date_initiated"] = state_subset.loc[state_subset['area'].isnull(),'date_initiated_y']

In [10]:
# Calculated "Stay at home" binary
final.drop("area",axis=1,inplace=True)

conditions = [
    (final['date'] < final['date_initiated']),
    (final['date'] >= final['date_initiated'])]
choices = [0,1]
final['SAH'] = np.select(conditions, choices)

final[["date",'county','state','cases','deaths','type','SAH']].to_csv('final.csv',index=False)